# Spacy/Medspacy IAA

## Resources

Prodigy forum answer about IAA for spans https://support.prodi.gy/t/proper-way-to-calculate-inter-annotator-agreement-for-spans-ner/5760

Spacy scorer object https://spacy.io/api/scorer

In [2]:
import spacy